Install and import required packages

In [1]:
!pip install --upgrade google-genai --quiet

In [2]:
from google import genai
from google.genai import types
import base64

The model, system prompt, and configurations including the safety settings can be set.

In [3]:
model = "gemini-2.0-flash-lite-001"

In [4]:
si_text1 = """Classify user questions into one of the following categories: Employment, General Information, Emergency Services, or Tax Related. Output only one of the following: Employment, General Information, Emergency Services, or Tax Related"""

generate_content_config = types.GenerateContentConfig(
    temperature = 1,
    top_p = 1,
    max_output_tokens = 8192,
    safety_settings = [types.SafetySetting(
      category="HARM_CATEGORY_HATE_SPEECH",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_DANGEROUS_CONTENT",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_SEXUALLY_EXPLICIT",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_HARASSMENT",
      threshold="OFF"
    )],

    system_instruction=[types.Part.from_text(text=si_text1)],
  )

The Client is invoked and the generate function is called with the user input.

In [5]:
def generate_classification(question):
  client = genai.Client(
      vertexai=True,
      project="qwiklabs-gcp-02-598c62ac8986",
      location="global",
  )
  response_text = ""
  for chunk in client.models.generate_content_stream(
    model = model,
    contents=[question],
    config = generate_content_config,
    ):
    print(chunk.text, end="")
    response_text += chunk.text
  return response_text

In [6]:
while True:
    try:
      question = input("User: ")
      if question.strip().lower() in ["exit", "no", "quit"]:
          print("Bye!")
          break
      else:
        generate_classification(question)
    except KeyboardInterrupt:
      print("\n\nGoodbye!")
      break

User: Are there any roles open?
Employment
User: expiry date of milk?
General Information


Goodbye!


##Unit test cases for the first function.
Testing deterministic values of the function whether it responds with the mentioned set of responses.

In [7]:
import unittest
class TestDeterministic(unittest.TestCase):
  def test_isEmployment(self):
    response = generate_classification("Are there any roles for government jobs?")
    self.assertEqual(response.strip(), "Employment")
  def test_isGeneral(self):
    response = generate_classification("What is the expiry date of milk bought today")
    self.assertEqual(response.strip(), "General Information")
  def test_isEmergency(self):
    response = generate_classification("What is the emergency helpline number")
    self.assertEqual(response.strip(), "Emergency Services")
  def test_isTax(self):
    response = generate_classification("My salary is 15k per month. How much tax do I have to pay?")
    self.assertEqual(response.strip(), "Tax Related")

unittest.main(argv=[''], verbosity=2, exit=False)

test_isEmergency (__main__.TestDeterministic.test_isEmergency) ... ok
test_isEmployment (__main__.TestDeterministic.test_isEmployment) ... 

Emergency Services


ok
test_isGeneral (__main__.TestDeterministic.test_isGeneral) ... 

Employment


ok
test_isTax (__main__.TestDeterministic.test_isTax) ... 

General Information


ok

----------------------------------------------------------------------
Ran 4 tests in 2.031s

OK


Tax Related


Second Function that generates social media posts for government announcements.

In [8]:
si_text2 = """Write a short, friendly social media post for government announcements like weather emergencies, holidays, school closings."""

generate_content_config_2 = types.GenerateContentConfig(
    temperature = 0.5,
    top_p = 1,
    max_output_tokens = 8192,
    safety_settings = [types.SafetySetting(
      category="HARM_CATEGORY_HATE_SPEECH",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_DANGEROUS_CONTENT",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_SEXUALLY_EXPLICIT",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_HARASSMENT",
      threshold="OFF"
    )],
    system_instruction=[types.Part.from_text(text=si_text2)],
  )

In [9]:
def generate_posts(question):
  client = genai.Client(
      vertexai=True,
      project="qwiklabs-gcp-02-598c62ac8986",
      location="global",
  )
  response_text=""
  for chunk in client.models.generate_content_stream(
    model = model,
    contents=[question],
    config = generate_content_config_2,
    ):
    print(chunk.text, end="")
    response_text += chunk.text
  return response_text

In [10]:
while True:
    try:
      question = input("User: ")
      if question.strip().lower() in ["exit", "no", "quit"]:
          print("Bye!")
          break
      else:
        generate_posts(question)
    except KeyboardInterrupt:
      print("\n\nGoodbye!")
      break

User: heavy rains in chennai
Stay safe, Chennai! 🌧️ Heavy rains are expected. Please take precautions, avoid unnecessary travel, and stay updated on the latest weather alerts. #ChennaiRains #StaySafe #WeatherAlert
User: christmas holidays from 25th to 2nd jan
Happy Holidays! 🎄 Our offices will be closed from December 25th to January 2nd for the Christmas break. We hope you have a safe and joyful holiday season! ✨ #ChristmasHolidays #HappyHolidays #GovernmentAnnouncement


Goodbye!


In [11]:
from vertexai.preview.generative_models import GenerativeModel

def does_post_follow_announcement_rules(post):
    model = GenerativeModel("gemini-2.0-flash-lite-001")
    prompt = f"""Does the following government announcement post follow these rules?
1. The post is short, friendly, and clear.
2. The topic is a government announcement like a weather emergency, holiday, or school closing.
Only answer Yes or No.

Post: {post}
Output:"""
    response = model.generate_content(prompt)
    return response.text.strip()

In [12]:
import unittest

class TestAnnouncementPostRelevance(unittest.TestCase):
    def test_announcement_post_relevance_true(self):
        post = generate_posts("heavy rains expected")
        isTrue = does_post_follow_announcement_rules(post)
        self.assertEqual(isTrue, "Yes")

unittest.main(argv=[''], verbosity=2, exit=False)


test_announcement_post_relevance_true (__main__.TestAnnouncementPostRelevance.test_announcement_post_relevance_true) ... 

Heads up, everyone! ☔️ We're expecting heavy rains today. Please stay safe on the roads and be mindful of potential flooding. Stay tuned for updates! #WeatherAlert #StaySafe #[YourCity/Region]


ok
test_isEmergency (__main__.TestDeterministic.test_isEmergency) ... ok
test_isEmployment (__main__.TestDeterministic.test_isEmployment) ... 

Emergency Services


ok
test_isGeneral (__main__.TestDeterministic.test_isGeneral) ... 

Employment


ok
test_isTax (__main__.TestDeterministic.test_isTax) ... 

General Information


ok

----------------------------------------------------------------------
Ran 5 tests in 3.583s

OK


Tax Related


Creating an Evaluation Dataset for function 1

In [13]:
import pandas as pd

eval_data = [
    {"prompt": "Are there any roles for government jobs?", "response": "Employment"},
    {"prompt": "What is the expiry date of milk bought today?", "response": "General Information"},
    {"prompt": "What is the emergency helpline number?", "response": "Emergency Services"},
    {"prompt": "My salary is 15k per month. How much tax do I pay?", "response": "Tax Related"},
    {"prompt": "All schools will be closed due to snow on Monday.", "response": "School Closing Announcement"}
]

eval_dataset = pd.DataFrame(eval_data)

Creating the Evaluation task

In [14]:
from vertexai.evaluation import EvalTask
import datetime
from vertexai.evaluation import (
    EvalTask,
    PointwiseMetric,
    PairwiseMetric,
    PointwiseMetricPromptTemplate,
    PairwiseMetricPromptTemplate,
    MetricPromptTemplateExamples,
)


metrics = [
    MetricPromptTemplateExamples.Pointwise.INSTRUCTION_FOLLOWING,
        MetricPromptTemplateExamples.Pointwise.GROUNDEDNESS,
        MetricPromptTemplateExamples.Pointwise.VERBOSITY,
        MetricPromptTemplateExamples.Pointwise.SUMMARIZATION_QUALITY
]

run_ts = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
experiment_name = f"gov-announcement-eval-{run_ts}"
eval_task = EvalTask(
    dataset=eval_dataset,
    metrics=metrics,
    experiment=experiment_name,
)

model = GenerativeModel("gemini-2.0-flash-lite-001")
result = eval_task.evaluate(
 experiment_run_name=experiment_name
 )

print("Summary metrics:", result.summary_metrics)
print(result.metrics_table)

INFO:vertexai.evaluation._evaluation:Computing metrics with a total of 20 Vertex Gen AI Evaluation Service API requests.
100%|██████████| 20/20 [00:02<00:00,  7.71it/s]
INFO:vertexai.evaluation._evaluation:All 20 metric requests are successfully computed.
INFO:vertexai.evaluation._evaluation:Evaluation Took:2.616478054002073 seconds


Summary metrics: {'row_count': 5, 'instruction_following/mean': np.float64(1.0), 'instruction_following/std': 0.0, 'groundedness/mean': np.float64(0.2), 'groundedness/std': 0.447213595499958, 'verbosity/mean': np.float64(-2.0), 'verbosity/std': 0.0, 'summarization_quality/mean': np.float64(1.6), 'summarization_quality/std': 0.8944271909999159}
                                              prompt  \
0           Are there any roles for government jobs?   
1      What is the expiry date of milk bought today?   
2             What is the emergency helpline number?   
3  My salary is 15k per month. How much tax do I ...   
4  All schools will be closed due to snow on Monday.   

                      response  \
0                   Employment   
1          General Information   
2           Emergency Services   
3                  Tax Related   
4  School Closing Announcement   

                   instruction_following/explanation  \
0  The response provides a very minimal answer th...   
